## Lab3: Molecular Dynamics

In this lab, we will prepare and run the necessary steps to perform molecular dynamics (MD) simulations of the cannabinoid 2 (CB2) receptor-ligand complex. 

The lab includes a series of steps to interact with the GROMACS program, as well as with a few Python scripts included within the Python package PyMemDyn. Normally, these steps are all run at once, but here we run them individually to better understand which steps are needed to successfully run MD on a membrane protein.

I recommend to read up on PyMemDyn and related webserver if the following papers: \
https://portlandpress.com/biochemsoctrans/article-abstract/41/1/205/68208/Characterization-of-the-dynamic-events-of-GPCRs-by \
https://academic.oup.com/nar/article/44/W1/W455/2499371?login=true

Below, you will find the relevant code blocks that you can execute by using the play button on top, or by typing Shift+Enter. You will use PyMOL to visualize the process of creating an MD simulation, and the MD simulation itself.

If you want to know more about what the code is doing, please ask one of the teachers!

### Load PyMemDyn

In [50]:
# Import python native functions
import os
import sys

# Add PyMemDyn modules
sys.path.insert(0, '../')

import settings
from pymemdyn import Run

### Input

We copy in the protein and ligand input files as these have already been pre-processed before the lab.

In [51]:
if not os.getcwd().endswith('rundir'):
    os.chdir('rundir')

In [52]:
# copy input files to working directory
!cp ../data/protein.pdb .
!cp ../data/E3R.pdb .
!cp ../data/E3R.itp .
!cp ../data/E3R.ff .

We use the crystal structure 6KPF (https://www.rcsb.org/structure/6KPF) as a starting point. By opening the protein.pdb file in PyMOL, you can see that the structure has been modified compared to the original crystal structure file on the RCSB PDB. 

Due to limitations in crystalizing highly-flexible regions of a protein, some sub-structures are not present within the crystal structure. Therefore, we also connect loops which were disconnected, and check if all amino acid side-chains are present. Also, the G protein was removed from the complex.


For the ligand input, we pre-generated the OPLS forcefield parameters using the LigParGen webserver (http://zarbi.chem.yale.edu/ligpargen/).



### Running PyMemDyn

First, we instantiate the Run class of PyMemDyn, this top level object takes run time variables and stores them in an object so we can use them in the next steps

In [53]:
run = Run(own_dir = os.getcwd(),
          repo_dir = settings.TEMPLATES_DIR,
          pdb = 'protein.pdb',
          ligand = 'E3R',
          alosteric = None,
          waters = None,
          ions = None,
          cho = None,
          ligpargen = None,
          library = None,
          restraint = 'bw',
          queue = "",
          debug = True
)

The command bellow returns a list of each of the steps run by the PyMemDyn.
We will use these keywords in the following command blocks to run the individual steps
in the program.
Note that PyMemDyn can also run everything in one go (this was the intention of the 
program), but for teaching purposes it makes more sense to carefully look at each individual step.

In [54]:
run.moldyn_notebook_info()

['pdb2gmx', 'set_itp', 'concat', 'editconf', 'set_protein_size', 'editconf2', 'set_protein_size2', 'set_popc', 'editconf3', 'make_ndx_lig', 'genrestr_lig', 'editconf4', 'make_topol', 'editconf5', 'solvate', 'set_water', 'editconf6', 'editconf7', 'solvate2', 'count_lipids', 'make_topol2', 'make_topol_lipids', 'make_ffoplsaanb', 'set_grompp', 'set_chains', 'make_ndx', 'grompp', 'trjconv', 'get_charge', 'genion', 'grompp2', 'trjconv2', 'grompp3', 'trjconv3', 'clean_pdb']


You can use the code block below to run each of the commands individually, there will be a short description of each of the
steps printed out. Some commands are running via GROMACS (indicated in the header) other commands are actual Python
scripts part of the PyMemDyn package.

Sometimes the help function mentions a filetype (.top or .itp for instance) if you don't know what these files are
you can find more information here: \
https://manual.gromacs.org/documentation/5.1.1/user-guide/file-formats.html


#### Setup Protein-Ligand Complex

In [55]:
run.moldyn_notebook_run("pdb2gmx")

LigandInit : pdb2gmx.
Running the following command
{'gromacs': 'pdb2gmx', 'options': {'src': 'protein-his.pdb', 'tgt': 'proteinopls.pdb', 'top': 'protein.top'}}

========================INFO============================

GROMACS command
 Convert the input pdb file to a gromacs object

========================INFO END========================


Using the Oplsaa force field in directory oplsaa.ff

going to rename oplsaa.ff/aminoacids.r2b
Reading /home/remco/apps/pymemdyn-teaching/notebook/rundir/protein-his.pdb...
Read '', 2326 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 299 residues with 2326 atoms

  chain  #res #atoms
  1 ' '   299   2326  

Reading residue database... (Oplsaa)
Processing chain 1 (2326 atoms, 299 residues)
Identified residue PRO21 as a starting terminus.
Identified residue LYS319 as a ending terminus.
Start terminus PRO-21: PRO-NH2+
End terminus LYS-319: COO-
Checking for dup

In [56]:
run.moldyn_notebook_run("set_itp")

LigandInit : set_itp.
Running the following command
{'command': 'set_itp', 'options': {'src': 'protein.top', 'tgt': 'protein.itp'}}

========================INFO============================

Python function (gromacs.py)
 Takes the generated .top file and stores it as itp file for later use

========================INFO END========================



In [57]:
run.moldyn_notebook_run("concat")

LigandInit : concat.
Running the following command
{'command': 'concat', 'options': {'src': 'proteinopls.pdb', 'tgt': <protein.ProteinComplex object at 0x7fc16059a8b0>}}

========================INFO============================

Python function (utils.py)
 Combines listed pdb files in one pdb file

========================INFO END========================



In [58]:
run.moldyn_notebook_run("editconf")

LigandInit : editconf.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'proteinopls.pdb', 'tgt': 'proteinopls.pdb', 'dist': 2.0}}

========================INFO============================

GROMACS command
 Converts generic structure format to .gro, .g96 or .pdb.

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 4816 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size :  4.440  5.759  7.098 (nm)
    center      : -0.724 -0.694  3.794 (nm)
    box vectors :  0.100  0.100  0.000 (nm)
    box angles  :   0.00   0.00  90.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  4.945  5.574  1.755 (nm)
new center      :  4.220  4.880  5.549 (nm)
new box vectors :  8.441  9.759 11.098 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 914.21               (nm^3)

If the molecule ro

In [59]:
run.moldyn_notebook_run("set_protein_size")

LigandInit : set_protein_size.
Running the following command
{'command': 'set_protein_size', 'options': {'src': 'proteinopls.pdb', 'dir': 'xy'}}

========================INFO============================

Python function (gromacs.py)
 Get the protein maximum base width from a pdb file (in xy direction)

========================INFO END========================



In [60]:
run.moldyn_notebook_run("editconf2")

LigandInit : editconf2.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'proteinopls.pdb', 'tgt': 'proteinopls.pdb', 'dist': 0.8}}

========================INFO============================

GROMACS command
 Re-adjust the previous system size with the above protein size

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 4816 atoms
Volume: 914.214 nm^3, corresponds to roughly 411300 electrons
No velocities found
    system size :  4.441  5.759  7.098 (nm)
    center      :  4.220  4.880  5.549 (nm)
    box vectors :  8.441  9.759 11.098 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 914.21               (nm^3)
    shift       : -1.200 -1.200 -1.200 (nm)
new center      :  3.020  3.680  4.349 (nm)
new box vectors :  6.041  7.359  8.698 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 386.68               (nm^3)



In [61]:
run.moldyn_notebook_run("set_protein_size2")

LigandInit : set_protein_size2.
Running the following command
{'command': 'set_protein_size', 'options': {'src': 'proteinopls.pdb', 'dir': 'z'}}

========================INFO============================

Python function (gromacs.py)
 Get the protein maximum length from a pdb file (in z direction)

========================INFO END========================



Now, the protein-ligand complex is essembled and positioned for insertion within the membrane. \
Open the proteinopls.pdb file in PyMOL to see what the complex looks like.

#### Embed Complex within Membrane

In [62]:
run.moldyn_notebook_run("set_popc")

LigandInit : set_popc.
Running the following command
{'command': 'set_popc', 'options': {'tgt': 'popc.pdb'}}

========================INFO============================

Python function (gromacs.py)
 Create a pdb file only with the lipid bilayer (POPC), no waters. 
Set some measures on the fly (height of the bilayer)

 != Have a look in PyMOL to see what this .pdb file looks like =!

========================INFO END========================



In [63]:
run.moldyn_notebook_run("editconf3")

LigandInit : editconf3.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'proteinopls.pdb', 'tgt': 'proteinopls.pdb', 'box': ['9.7594', '9.7594', '6.42'], 'angles': ['90', '90', '120'], 'bt': 'tric'}}

========================INFO============================

GROMACS command
 Adjust the box size of the system to take into consideration the POPC bilayer

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 4816 atoms
Volume: 386.678 nm^3, corresponds to roughly 174000 electrons
No velocities found
    system size :  4.440  5.759  7.098 (nm)
    center      :  3.020  3.680  4.349 (nm)
    box vectors :  6.041  7.359  8.698 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 386.68               (nm^3)
    shift       : -0.580  0.546 -1.139 (nm)
new center      :  2.440  4.226  3.210 (nm)
new box vectors :  9.759  9.759  6.420 (nm)
new box angl

In [64]:
run.moldyn_notebook_run("make_ndx_lig")

LigandInit : make_ndx_lig.
Running the following command
{'gromacs': 'make_ndx', 'options': {'src': 'E3R.pdb', 'tgt': 'ligand_ha.ndx', 'ligand': True}, 'input': '! a H*\nq\n'}

========================INFO============================

GROMACS command
 Generate ndx file for the ligand

========================INFO END========================

Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :    58 atoms
  1 Other               :    58 atoms
  2 LIG                 :    58 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "name": group             'case': case sensitive           'q': save a

In [65]:
run.moldyn_notebook_run("genrestr_lig")

LigandInit : genrestr_lig.
Running the following command
{'gromacs': 'genrestr', 'options': {'src': 'E3R.pdb', 'tgt': 'posre_lig.itp', 'index': 'ligand_ha.ndx', 'forces': ['1000', '1000', '1000']}, 'input': '2\n'}

========================INFO============================

GROMACS command
 Add positional restrains for the ligand

========================INFO END========================

Select group to position restrain
Selected 2: 'LIG'



In [66]:
run.moldyn_notebook_run("editconf4")

LigandInit : editconf4.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'popc.pdb', 'tgt': 'popc.gro', 'box': ['6.1842', '6.1842', '6.42']}}

========================INFO============================

GROMACS command
 Create .gro file of the lipid bilayer

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 6656 atoms
No velocities found
    system size :  7.786  8.618  5.778 (nm)
    center      :  3.322  3.129  7.639 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       : -0.230 -0.037 -4.429 (nm)
new center      :  3.092  3.092  3.210 (nm)
new box vectors :  6.184  6.184  6.420 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 245.53               (nm^3)



In [67]:
run.moldyn_notebook_run("make_topol")

LigandInit : make_topol.
Running the following command
{'command': 'make_topol', 'options': {'complex': <protein.ProteinComplex object at 0x7fc16059a8b0>}}

========================INFO============================

Python function (gromacs.py)
 Add lipid positions to topol.top

========================INFO END========================



In [68]:
run.moldyn_notebook_run("editconf5")

LigandInit : editconf5.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'proteinopls.pdb', 'tgt': 'proteinopls.pdb', 'translate': ['0', '0', '0']}}

========================INFO============================

GROMACS command
 Check if any adjustments need to be made with the new lipids added

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 4816 atoms
Volume: 529.556 nm^3, corresponds to roughly 238300 electrons
No velocities found
Translating 4816 atoms (out of 4816) by 0 0 0 nm



In [69]:
run.moldyn_notebook_run("solvate")

LigandInit : solvate.
Running the following command
{'gromacs': 'solvate', 'options': {'cp': 'proteinopls.pdb', 'cs': 'popc.gro', 'tgt': 'protpopc.pdb', 'top': 'topol.top'}}

========================INFO============================

GROMACS command
 Insert the protein-ligand complex into the lipid bilayer and 
adding POP definitions in the topol.top file

 != Have a look in PyMOL to see what this .pdb file looks like =!"

========================INFO END========================


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. 

Now, the protein-ligand complex is embedded within the lipid bilayer. \
Open the protpopc.pdb file in PyMOL to see what the system now looks like.

#### Solvate Protein-Membrane System with Waters

In [70]:
run.moldyn_notebook_run("set_water")

LigandInit : set_water.
Running the following command
{'command': 'set_water', 'options': {'tgt': 'water.pdb'}}

========================INFO============================

Python function (gromacs.py)
 Create a water layer for a box 

!= Have a look in PyMOL to see what this .pdb file looks like =!

========================INFO END========================



In [71]:
run.moldyn_notebook_run("editconf6")

LigandInit : editconf6.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'water.pdb', 'tgt': 'water.gro', 'box': ['6.1842', '6.1842', '8.6983']}}

========================INFO============================

GROMACS command
 Create a box for the water molecules

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 18009 atoms
No velocities found
    system size :  6.324  6.325  8.867 (nm)
    center      :  3.058  3.062  7.753 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  0.034  0.030 -3.404 (nm)
new center      :  3.092  3.092  4.349 (nm)
new box vectors :  6.184  6.184  8.698 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 332.66               (nm^3)



In [72]:
run.moldyn_notebook_run("editconf7")

LigandInit : editconf7.
Running the following command
{'gromacs': 'editconf', 'options': {'src': 'protpopc.pdb', 'tgt': 'protpopc.pdb', 'box': ['9.7594', '9.7594', '8.6983'], 'angles': ['90', '90', '120'], 'bt': 'tric'}}

========================INFO============================

GROMACS command
 Adjust the protein/POPC box for the waters

========================INFO END========================

Note that major changes are planned in future for editconf, to improve usability and utility.Read 13292 atoms
Volume: 529.556 nm^3, corresponds to roughly 238300 electrons
No velocities found
    system size : 10.192 11.294  7.098 (nm)
    center      :  3.993  4.247  3.261 (nm)
    box vectors :  9.759  9.759  6.420 (nm)
    box angles  :  90.00  90.00 120.00 (degrees)
    box volume  : 529.56               (nm^3)
    shift       : -1.553 -0.021  1.088 (nm)
new center      :  2.440  4.226  4.349 (nm)
new box vectors :  9.759  9.759  8.698 (nm)
new box angles  :  90.00  90.00 120.00 (degrees)
n

In [73]:
run.moldyn_notebook_run("solvate2")

LigandInit : solvate2.
Running the following command
{'gromacs': 'solvate', 'options': {'cp': 'protpopc.pdb', 'cs': 'water.gro', 'tgt': 'tmp.pdb', 'top': 'topol.top'}}

========================INFO============================

GROMACS command
 Solvate the protein/POPC box with the box of waters

 != Have a look in PyMOL to see what this .pdb file looks like =!"

========================INFO END========================


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964

Now, the protein-membrane system is solvated with waters. \
Open the tmp.pdb file in PyMOL to see what the complex now looks like.

#### Prepare MD (1)

In [74]:
run.moldyn_notebook_run("count_lipids")

LigandInit : count_lipids.
Running the following command
{'command': 'count_lipids', 'options': {'src': 'tmp.pdb', 'tgt': 'popc.pdb'}}

========================INFO============================

Python function (gromacs.py)
 Counts the lipids in popc and makes atom tags (N4) for bookkeeping

========================INFO END========================



In [75]:
run.moldyn_notebook_run("make_topol2")

LigandInit : make_topol2.
Running the following command
{'command': 'make_topol', 'options': {'complex': <protein.ProteinComplex object at 0x7fc16059a8b0>}}

========================INFO============================

Python function (utils.py)
 Adjust the old topol.top file with the new topology

========================INFO END========================



In [76]:
run.moldyn_notebook_run("make_topol_lipids")

LigandInit : make_topol_lipids.
Running the following command
{'command': 'make_topol_lipids'}

========================INFO============================

Python function (gromacs.py)
 Add lipid positions to topol.top

========================INFO END========================



In [77]:
run.moldyn_notebook_run("make_ffoplsaanb")

LigandInit : make_ffoplsaanb.
Running the following command
{'command': 'make_ffoplsaanb', 'options': {'complex': <protein.ProteinComplex object at 0x7fc16059a8b0>}}

========================INFO============================

Python function (utils.py)
 Join all OPLS force fields needed to run the simulation

========================INFO END========================



In [78]:
run.moldyn_notebook_run("set_grompp")

LigandInit : set_grompp.
Running the following command
{'command': 'set_grompp', 'options': {'steep.mdp': 'steep.mdp', 'popc.itp': 'popc.itp', 'spc.itp': 'spc.itp', 'ions.itp': 'ions.itp', 'ffoplsaabon_mod.itp': 'ffoplsaabon_mod.itp', 'ffoplsaa_mod.itp': 'ffoplsaa_mod.itp'}}

========================INFO============================

Python function (gromacs.py)
 Copy template files in templates folder to working dir

========================INFO END========================



In [79]:
run.moldyn_notebook_run("set_chains")

LigandInit : set_chains.
Running the following command
{'command': 'set_chains', 'options': {'src': 'proteinopls.pdb'}}

========================INFO============================

Python function (gromacs.py)
 If we are dealing with a GPCR dimer we need to correct atom numbering after protonation

========================INFO END========================



In [80]:
run.moldyn_notebook_run("make_ndx")

LigandInit : make_ndx.
Running the following command
{'command': 'make_ndx', 'options': {'src': 'tmp.pdb', 'tgt': 'index.ndx'}}

========================INFO============================

GROMACS command
 Make GROMACS index file, which is useful for bookkeeping (which atom is what)

========================INFO END========================

Going to read 0 old index file(s)
Analysing residue names:
There are:   299    Protein residues
There are:   164      Other residues
There are: 11363      Water residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 47381 atoms
  1 Protein             :  4758 atoms
  2 Protein-H           :  2327 atoms
  3 C-alpha             :   299 atoms
  4 Backbone            :   897 atoms
  5 MainChain           :  1197 atoms
  6 MainChain+Cb        :  1480 atoms
  7 MainChain+H         :  1485 atoms
  8 SideChain           :  3273 atoms
  9 SideChain-H         :  1130 atoms

In [81]:
run.moldyn_notebook_run("grompp")

LigandInit : grompp.
Running the following command
{'gromacs': 'grompp', 'options': {'src': 'steep.mdp', 'src2': 'tmp.pdb', 'tgt': 'topol.tpr', 'top': 'topol.top', 'index': 'index.ndx'}}

========================INFO============================

GROMACS command
 gmx grompp (the gromacs preprocessor) reads a molecular topology file, checks the validity of the file, expands the topology from a molecular description to an atomic description.

========================INFO END========================


++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 84x84x80, spacing 0.116 0.116 0.109
This run will generate roughly 4 Mb of data



In [82]:
run.moldyn_notebook_run("trjconv")

LigandInit : trjconv.
Running the following command
{'gromacs': 'trjconv', 'options': {'src': 'tmp.pdb', 'src2': 'topol.tpr', 'tgt': 'tmp.pdb', 'pbc': 'mol', 'index': 'index.ndx'}, 'input': '1\n0\n'}

========================INFO============================

GROMACS command
 Can convert trajectory files, in this case is used to make a new topology file based on tmp.pdb and topol.tpr

========================INFO END========================

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'



#### Add Ions to System


In [83]:
run.moldyn_notebook_run("get_charge")

LigandInit : get_charge.
Running the following command
{'command': 'get_charge', 'options': {'src': 'steep.mdp', 'src2': 'tmp.pdb', 'tgt': 'topol.tpr', 'top': 'topol.top', 'index': 'index.ndx'}}

========================INFO============================

Python function (gromacs.py)
 Gets the total charge of a system using gromacs grompp command 

========================INFO END========================



In [84]:
run.moldyn_notebook_run("genion")

LigandInit : genion.
Running the following command
{'gromacs': 'genion', 'options': {'src': 'topol.tpr', 'tgt': 'output.pdb', 'src2': 'topol.top', 'index': 'index.ndx', 'np': 0, 'nn': 14}, 'input': ' SOL \n'}

========================INFO============================

GROMACS command
 Randomly replaces solvent molecules with monoatomic ions.

========================INFO END========================

Will try to add 0 NA+ ions and 14 CL- ions.
Select a continuous group of solvent molecules
Selected 17: 'SOL'

Processing topology
Replacing 14 solute molecules in topology file (/home/remco/apps/pymemdyn-teaching/notebook/rundir/topol.top)  by 0 NA+ and 14 CL- ions.



Now, the complex is made neutral in charge by adding ions. \
Open the output.pdb file in PyMOL to see what the system now looks like.

#### Prepare MD (2)

In [85]:
run.moldyn_notebook_run("grompp2")

LigandInit : grompp2.
Running the following command
{'gromacs': 'grompp', 'options': {'src': 'steep.mdp', 'src2': 'output.pdb', 'tgt': 'topol.tpr', 'top': 'topol.top'}}

========================INFO============================

GROMACS command
 gmx grompp (the gromacs preprocessor) reads a molecular topology file, checks the validity of the file, expands the topology from a molecular description to an atomic description.

========================INFO END========================


++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:   299    Protein residues
There are:   164      Other residues
There are: 11349      Water residues
There are:    14        Ion residues
Analysing Protein...
Analysing residues not c

In [86]:
run.moldyn_notebook_run("trjconv2")

LigandInit : trjconv2.
Running the following command
{'gromacs': 'trjconv', 'options': {'src': 'output.pdb', 'src2': 'topol.tpr', 'tgt': 'output.pdb', 'trans': [0, 0, 0], 'pbc': 'mol'}, 'input': '0\n'}

========================INFO============================

GROMACS command
 Can convert trajectory files, in this case is used to make a new topology file based on putput.pdb and topol.tpr

========================INFO END========================

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'



In [87]:
run.moldyn_notebook_run("grompp3")

LigandInit : grompp3.
Running the following command
{'gromacs': 'grompp', 'options': {'src': 'steep.mdp', 'src2': 'output.pdb', 'tgt': 'topol.tpr', 'top': 'topol.top'}}

========================INFO============================

GROMACS command
 gmx grompp (the gromacs preprocessor)

========================INFO END========================


++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:   299    Protein residues
There are:   164      Other residues
There are: 11349      Water residues
There are:    14        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into 

In [88]:
run.moldyn_notebook_run("trjconv3")

LigandInit : trjconv3.
Running the following command
{'gromacs': 'trjconv', 'options': {'src': 'output.pdb', 'src2': 'topol.tpr', 'tgt': 'hexagon.pdb', 'ur': 'compact', 'pbc': 'mol'}, 'input': '1\n0\n'}

========================INFO============================

GROMACS command
 Can convert trajectory files, in this case creates hexagon.pdb, which contains the whole system for further refinement

========================INFO END========================

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'



In [89]:
run.moldyn_notebook_run("clean_pdb")

LigandInit : clean_pdb.
Running the following command
{'command': 'clean_pdb', 'options': {'src': 'hexagon.pdb', 'tgt': 'hexagon.pdb'}}

========================INFO============================

Python function
 Remove incorrectly allocated atom identifiers in pdb file

 != Have a look in PyMOL to see what this .pdb file looks like =!"

========================INFO END========================



The final system for MD simulation has now been created. The system has now been converted into an hexagonal shape. \
Open the hexagon.pdb file in PyMOL to see what the system now looks like.

### MD Simulation

**After** you have completed all steps above, you would normally start running the MD simulation within PyMemDyn. However it takes a few hours to complete on a pretty large cluster,
so we have already done it for you.
The output from a PyMemDyn run can be found in the finalOutput folder, which we copy in here now.

In [90]:
!cp -r ../data/finalOutput .

Typically, it's useful to analyze a few properties, like temperature and pressure, but also RMSD and RMSF (if you don't know what those are please Google it or ask a TA :) ) are good indicators for the stability of your simulation. We can use a basic plotting program called Grace to work with GROMACS output files, e.g.: to check the temperature during your simulation run:

In [92]:
!xmgrace finalOutput/reports/temp.xvg

/bin/bash: xmgrace: command not found


In [ ]:
!xmgrace finalOutput/reports/temp2.xvg

In [ ]:
!xmgrace finalOutput/reports/rmsd-all-atom-vs-start.xvg

Plot the other properties and RMSD/RMSF in the /reports folder as well (all the other .xvg files), and save them in a file. Note that the properties are seperated into two files, each for a specific part of the simulation. Write a short description for each of the figures. Discuss what you are seeing with your fellow students and/or TAs. Note that you need to close the xmgrace window that opened everytime you want to load a new figure, or you need to add a new code block.

Next, we can have a look at the trajectory. Open PyMOL and change to the finalOutput directory. There, you can load a PyMOL script (.pml) generated by PyMemDyn, by typing @load_gpcr.pml

- Can you see when the restraints are released in the simulation? 
- Can you link that to events described in the PyMemDyn and GPCR-ModSim papers?
- Do you think the ligand looks stable?
- What could you do to further assess ligand stability?
- Do we know anything about the binding affinity now?

Thanks for participating in this last computerlab, as always please ask your TA any questions if something is unclear!